In [1]:
from preprocess import *

Using backend: pytorch


In [2]:
datapath

'/home/zhangkechi/workspace/data/codenet/Project_CodeNet_C++1400_spts/'

In [3]:
trainset, devset, testset, token_vocabsize, type_vocabsize = get_spt_dataset()

420000 420000 420000
197838050
198258050
198258050


KeyboardInterrupt: 

In [4]:
len(trainset)

NameError: name 'trainset' is not defined

In [4]:
trainset[0]

[{'graph': Graph(num_nodes=485, num_edges=484,
        ndata_schemes={'token': Scheme(shape=(), dtype=torch.int64), 'type': Scheme(shape=(), dtype=torch.int64), 'depth': Scheme(shape=(), dtype=torch.int64), 'dfs': Scheme(shape=(), dtype=torch.int64)}
        edata_schemes={}),
  'edge': [[0, 1],
   [0, 2],
   [1, 3],
   [1, 4],
   [1, 5],
   [1, 6],
   [1, 7],
   [1, 8],
   [1, 9],
   [1, 10],
   [1, 11],
   [1, 12],
   [1, 13],
   [1, 14],
   [1, 15],
   [3, 16],
   [3, 17],
   [3, 18],
   [3, 19],
   [4, 20],
   [4, 21],
   [5, 22],
   [5, 23],
   [6, 24],
   [6, 25],
   [7, 26],
   [7, 27],
   [8, 28],
   [8, 29],
   [8, 30],
   [9, 31],
   [9, 32],
   [9, 33],
   [10, 34],
   [10, 35],
   [10, 36],
   [11, 37],
   [11, 38],
   [11, 39],
   [12, 40],
   [12, 41],
   [12, 42],
   [12, 43],
   [12, 44],
   [13, 45],
   [13, 46],
   [13, 47],
   [13, 48],
   [13, 49],
   [14, 50],
   [14, 51],
   [14, 52],
   [14, 53],
   [14, 54],
   [15, 55],
   [15, 56],
   [15, 57],
   [20, 58],
  

In [5]:
def graph2tree(edge, token_tensor, type_tensor, depth_tensor, dfs_tensor):
    root = []
    children_dict = {}
    for e in edge:
        if e[0] in children_dict:
            children_dict[e[0]].append(e[1])
        else:
            children_dict[e[0]] = []
            children_dict[e[0]].append(e[1])
    
    def addNode(tree, this_id):
        tree.append({})
        this_token = token_tensor[this_id]
        this_type = type_tensor[this_id]
        this_depth = depth_tensor[this_id]
        this_dfs = dfs_tensor[this_id]
        
        tree[-1]['node'] = this_token
        tree[-1]['type'] = this_type
        tree[-1]['depth'] = this_depth
        tree[-1]['dfs'] = this_dfs
        tree[-1]['children'] = []
        
        if this_id not in children_dict:
            return
        for e in children_dict[this_id]:
            addNode(tree[-1]['children'], e)
    
    addNode(root, 0)
    
    return root

def getTree(data):
    edge = data[0]['edge']
    token_tensor = data[0]['graph'].ndata['token'].tolist()
    type_tensor = data[0]['graph'].ndata['type'].tolist()
    depth_tensor = data[0]['graph'].ndata['depth'].tolist()
    dfs_tensor = data[0]['graph'].ndata['dfs'].tolist()
    return graph2tree(edge, token_tensor, type_tensor, depth_tensor, dfs_tensor)


In [6]:
from tqdm.notebook import tqdm

In [7]:
def dataset_convert(dataset):
    output_dataset = []
    labels = []
    for e in tqdm(dataset):
        t = getTree(e)[0]
        l = e[1]
        d = {'tree':t, 'label':l}
        labels.append(l)
        output_dataset.append(d)
    return output_dataset, list(set(labels))    

In [12]:
total_len = len(trainset)

In [13]:
total_len

252000

做数据集

In [19]:
trees, labels = dataset_convert(trainset[int(total_len/2):])
data = (trees, labels)

MemoryError: 

In [15]:
len(trees)

126000

In [16]:
len(labels)

700

In [17]:

save_path = '/home/zhangkechi/workspace/treecaps-master/codenet_data/Project_CodeNet_C++1400_spts/train2.pkl'
import pickle
with open(save_path, 'wb') as file_handler:
    pickle.dump(data, file_handler, protocol=pickle.HIGHEST_PROTOCOL)